In [11]:

# Computation of the orbit model the best fit potential in Malhan+19 (MWPotential2014+axisymmetry), 
# using Malhan+19 data.
# Optimization to find best fit orbit.

from astropy import units as u
from astropy.coordinates import SkyCoord
import astropy.coordinates as coord
from scipy.integrate import solve_ivp
from astropy.io import ascii
from scipy import optimize
from scipy import interpolate
import numpy as np
import scipy.integrate as integrate
from scipy.interpolate import InterpolatedUnivariateSpline
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
import config as cfg
import GD1Koposov10_class as GD1_class
import pandas as pd
from astroquery.gaia import Gaia
import galpy as gp
from galpy.potential.mwpotentials import MWPotential2014
from galpy.potential import TriaxialNFWPotential
from galpy.orbit import Orbit


# Orbit definition
# The orbit_model here defined works with sky coordinates at input and sky-cartesian at output
def orbit_model(alpha,delta,distance,mu_alpha,mu_delta,v_los):
#    print('param= ',alpha,delta,distance,mu_alpha,mu_delta,v_los)
    
    # Transformation to galactocentric coordinates
    sky_coord = coord.ICRS(ra=alpha*u.degree, dec=delta*u.degree,
                distance=distance*u.kpc,
                pm_ra_cosdec=mu_alpha*np.cos(delta*u.degree)*u.mas/u.yr,
                pm_dec=mu_delta*u.mas/u.yr,
                radial_velocity=v_los*u.km/u.s)
    galcen_distance = 8.129*u.kpc
    v_sun = coord.CartesianDifferential([11.1, 229.0+12.24, 7.25]*u.km/u.s)
    z_sun=0.0*u.kpc
    frame = coord.Galactocentric(galcen_distance=galcen_distance,
                                galcen_v_sun=v_sun,
                                z_sun=z_sun)
    galac_coord= sky_coord.transform_to(frame)
    
    w_0 = np.zeros(6)
    w_0[:3]=[galac_coord.x/u.kpc,galac_coord.y/u.kpc,galac_coord.z/u.kpc]
    w_0[3:]=[galac_coord.v_x/(u.km/u.s),galac_coord.v_y/(u.km/u.s),galac_coord.v_z/(u.km/u.s)]

    from astropy.coordinates import SkyCoord
    import astropy.units as u
    c= SkyCoord(ra=20.*u.deg,dec=30.*u.deg,distance=2.*u.kpc,
                pm_ra_cosdec=-10.*u.mas/u.yr,pm_dec=20.*u.mas/u.yr,
                radial_velocity=50.*u.km/u.s)
    o=Orbit(c)
    print('orbita1=',o)
    o= Orbit(sky_coord)
    print('orbita2=',o)
    
    # ODE integration
    unit_t = 0.977792221680356   # Gyr
    time_span_s2 = 0.2/unit_t #
    t_0=0.0/unit_t    
    n_steps = 1000
    t_back = np.linspace(t_0,-time_span_s2, n_steps+1)
    t_forw = np.linspace(t_0,time_span_s2, n_steps+1)       
    sol_back = solve_ivp(symp_grad_mw, [t_0,-time_span_s2], w_0, t_eval=t_back,method='DOP853',rtol=5.0e-14,atol=0.5e-14)
    sol_forw = solve_ivp(symp_grad_mw, [t_0,time_span_s2], w_0, t_eval=t_forw,method='DOP853',rtol=5.0e-14,atol=0.5e-14)
    
    t = np.concatenate([sol_back.t,sol_forw.t])
    y = np.concatenate([sol_back.y, sol_forw.y],axis=1)
    y = np.delete(y,0,axis=1) #Remove duplicated column
    
    #Transformation to GD-1 frame of coordinates (\phi_1, \phi_2)
    galac_coord=coord.Galactocentric(x=y[0]*u.kpc,y=y[1]*u.kpc,z=y[2]*u.kpc,
                                     v_x=y[3]*u.km/u.s,v_y=y[4]*u.km/u.s,v_z=y[5]*u.km/u.s,
                           galcen_distance=galcen_distance,galcen_v_sun=v_sun,z_sun=z_sun) 
    gd1_coord = galac_coord.transform_to(GD1_class.GD1Koposov10)
    phi_1 = gd1_coord.phi1
    phi_2 = gd1_coord.phi2
    d_hel = gd1_coord.distance
    v_hel = gd1_coord.radial_velocity
    mu_phi_1 = gd1_coord.pm_phi1_cosphi2/np.cos(phi_2)  #not used by Ibata
    mu_phi_2 = gd1_coord.pm_phi2
    #return phi_1, phi_2, d_hel, v_hel, mu_phi_1, mu_phi_2
    # Transformation to ICRS coordinates      
    icrs_coord=galac_coord.transform_to(coord.ICRS)
    mu_ra = icrs_coord.pm_ra_cosdec / np.cos(icrs_coord.dec)
    mu_dec= icrs_coord.pm_dec
    return phi_1, phi_2, d_hel, mu_ra, mu_dec, v_hel, y[0], y[1], y[2]


# Ibata's polynomials.
# [x] = radians
# [S] = radians
# [D] = kpc
# [V] = km/s                                                              
# [MU] = mas/year
class IbaPoly:
    def S(self,x):
        return 0.008367*x**3-0.05332*x**2-0.07739*x-0.02007
    def D(self,x):
        return -4.302*x**5-11.54*x**4-7.161*x**3+5.985*x**2+8.595*x+10.36
    def V(self,x):
        return 90.68*x**3+204.5*x**2-254.2*x-261.5
    def MU_RA(self,x):
        return 3.794*x**3+9.467*x**2+1.615*x-7.844
    def MU_DEC(self,x):
        return -1.225*x**3+8.313*x**2+18.68*x-3.95
    limit = [-90,10]

pol = IbaPoly()
print('valor=',pol.S(np.linspace(-90,10,10)))

# Observations (Ibata polynomials evaluated in a grid)
Iba_sky = pd.DataFrame()
Iba_sky['phi_1'] = np.linspace(IbaPoly.limit[0],IbaPoly.limit[1],100)
Iba_sky['phi_2'] = pol.S(cfg.deg2rad*Iba_sky['phi_1'])/cfg.deg2rad
Iba_sky['d_hel'] = pol.D(cfg.deg2rad*Iba_sky['phi_1'])
Iba_sky['v_hel'] = pol.V(cfg.deg2rad*Iba_sky['phi_1'])
Iba_sky['mu_ra'] = pol.MU_RA(cfg.deg2rad*Iba_sky['phi_1'])
Iba_sky['mu_dec'] = pol.MU_DEC(cfg.deg2rad*Iba_sky['phi_1'])

def invert_ic(u_0):
    w_0=u_0
    gd1_coord = GD1_class.GD1Koposov10(phi1=u_0[0]*u.degree, phi2=u_0[1]*u.degree)
    icrs_coord = gd1_coord.transform_to(coord.ICRS)
    w_0[0]=icrs_coord.ra.value
    w_0[1]=icrs_coord.dec.value
    return w_0   

#print('To compare with the Figures in Malhan+19')
[Iba_sky['RA'],Iba_sky['Dec']] = invert_ic([Iba_sky['phi_1'],Iba_sky['phi_2']])


# Load Malhan+19 tables
"""
Uncomment to download data from Gaia for the first time. 
Otherwise just open the file at the bottom of this cell.

data = cross_match('asu.tsv')
"""

# Open cross matched file
from astropy.table import Table
data = Table.read('cross_matched.dat', format='ascii')
print(type(data))
print(len(data))

arr = data['Pmember']
boolarr=  (arr=='Y')

data = data[boolarr]
print(type(data))
print(len(data))



# Chi^2 function
def chi2(w_0):
	import wrap_ra as wrap
    
	phi_1, phi_2, d_hel, mu_ra, mu_dec, v_hel, x,y,z = orbit_model(w_0[0],w_0[1],w_0[2],w_0[3],w_0[4],w_0[5]) 
    
	[ra,dec] = invert_ic([phi_1.value,phi_2.value])
	cfg.dec_spl = interp1d(ra,dec,kind='cubic')
	cfg.v_hel_spl = interp1d(ra,v_hel,kind='cubic')
	cfg.mu_ra_spl  = interp1d(ra,mu_ra,kind='cubic')
	cfg.mu_dec_spl  = interp1d(ra,mu_dec,kind='cubic')
    	
	cfg.ra_min = np.amin(ra)
	cfg.ra_max = np.amax(ra)
  
	sum=np.zeros(4)
	y_mod = wrap.dec_wrap(data['RAJ2000'])
	y_dat = data['DEJ2000']
	sigma2 = 0.01**2
	sum[0] = np.sum( (y_dat-y_mod)**2 / sigma2 )
	
	y_mod = wrap.mu_ra_wrap(data['RAJ2000'])
	y_dat = data['pmra']
	sigma2 = data['pmra_error']**2
	sum[1] = np.sum( (y_dat-y_mod)**2 / sigma2 )
	
	y_mod = wrap.mu_dec_wrap(data['RAJ2000'])
	y_dat = data['pmdec']
	sigma2 = data['pmdec_error']**2
	sum[2] = np.sum( (y_dat-y_mod)**2 / sigma2 )
    
	y_mod = wrap.v_hel_wrap(data['RAJ2000'])
	y_dat = data['Vlos']
	sigma2 = data['e_Vlos']**2
	sum[3] = np.sum( (y_dat-y_mod)**2 / sigma2 )
	
	print('chi^2 =',np.sum(sum))
	return np.sum(sum)


# Optimization
#----------------

# Potential selection:
mw = MWPotential2014
mw[2]=TriaxialNFWPotential(a=16.0*u.kpc,b=1.0,c=0.82,normalize=0.59)
print('mw=',mw)
for i in range(0,3):
    mw[i].turn_physical_on()
r_sun=8.122*u.kpc
v0=mw[0].vcirc(r_sun)
v1=mw[1].vcirc(r_sun)
v2=mw[2].vcirc(r_sun)
v_circ=np.sqrt(v0*v0+v1*v1+v2*v2)
print(v_circ)


valor= [-6.52448997e+03 -4.43362521e+03 -2.84486168e+03 -1.68933519e+03
 -8.98181542e+02 -4.02536534e+02 -1.33535970e+02 -2.23156531e+01
 -1.13856379e-02  2.24103000e+00]
<class 'astropy.table.table.Table'>
97
<class 'astropy.table.table.Table'>
68
mw= [<galpy.potential.PowerSphericalPotentialwCutoff.PowerSphericalPotentialwCutoff object at 0x7fe713932820>, <galpy.potential.MiyamotoNagaiPotential.MiyamotoNagaiPotential object at 0x7fe713932850>, <galpy.potential.TwoPowerTriaxialPotential.TriaxialNFWPotential object at 0x7fe70f640880>]
244.86405449288077 km / s
